# Traitement du Langage Naturel (NLP): 
#          Réseaux Récurrents (RNNs)

Le traitement du langage naturel en pratique suit le même principe que le domaine de vision. C'est-à-dire en utilisant le "l'apprentissage par transfert".

Cependant le type de tâche de pré-entraînement est très différent de celui de la tâche cible. On utilise l'apprentissage auto-supervisé.
Le résultat de cet apprentissage s'appelle: **un modèle du langage**.
La tâche cible à laquelle nous nous intéresserons est l'*analyse du sentiment*.

Au niveau de l'apprentissage auto-supervisé, aucun label n'est utilisé contrairement à la vision, on utilise des labels pour une première tâche.

En pratique, nous apprenons notre réseau à deviner le prochain mot en lui fournissant les premiers mots débutant une phrase.

Ainsi l'aptitude à compléter ue phrase par un mot implique une connaissance de la langue et une compréhension contextuelle. Et cette connaissance peut être utiliser pour diverses tâches basées sur la compréhension d'un corpus en l'occurrence,l'analyse de sentiments.

L'apprentissage auto-supervisé prend de l'ampleur à présent dans la vision également où le réseau apprend à reconstituer une partie manquante d'une image, voir 



> jargon: Apprentissage auto-supervisé: Entraîner un modèle à l'aide d'étiquettes incorporées dans la variable indépendante, plutôt que d'exiger des étiquettes externes. Par exemple, entraîner un modèle pour prédire le mot suivant dans un texte.

L'anglais de Wikipedia est légèrement différent de l'anglais IMDb, donc au lieu de passer directement au classifieur, nous pourrions affiner notre modèle de langage pré-entraîné au corpus IMDb, puis l'utiliser comme base de notre classifieur.

Même si notre modèle de langage connaît les bases de la langue que nous utilisons dans la tâche (par exemple, notre modèle pré-entraîné est en anglais), cela aide à s'habituer au style du corpus que nous ciblons. Il peut s'agir d'un langage plus informel, ou plus technique, avec de nouveaux mots à apprendre ou différentes manières de composer des phrases. Dans le cas de l'ensemble de données IMDb, il y aura beaucoup de noms de réalisateurs et d'acteurs de films, et souvent un style de langage moins formel que celui vu sur Wikipedia.

Nous avons déjà vu qu'avec fastai, nous pouvons télécharger un modèle de langue anglaise pré-entraîné et l'utiliser pour obtenir des résultats de pointe pour la classification PNL. (Nous prévoyons que des modèles pré-entraînés dans de nombreuses autres langues seront bientôt disponibles - ils pourraient bien être disponibles au moment où vous lirez ce livre, en fait.) Alors, pourquoi apprenons-nous à former un modèle de langage en détail?

Une des raisons, bien sûr, est qu'il est utile de comprendre les fondements des modèles que vous utilisez. Mais il y a une autre raison très pratique, qui est que vous obtenez des résultats encore meilleurs si vous affinez le modèle de langage (basé sur des séquences) avant de peaufiner le modèle de classification. Par exemple, pour la tâche d'analyse des sentiments IMDb, l'ensemble de données comprend 50 000 critiques de films supplémentaires auxquelles aucun libellé positif ou négatif n'est associé. Puisqu'il y a 25 000 critiques étiquetées dans l'ensemble d'entraînement et 25 000 dans l'ensemble de validation, cela fait 100 000 critiques de films au total. Nous pouvons utiliser toutes ces critiques pour affiner le modèle de langage pré-entraîné, qui n'a été entraîné que sur des articles de Wikipédia; il en résultera un modèle de langage particulièrement efficace pour prédire le prochain mot d'une critique de film.

C'est ce qu'on appelle l'approche ULMFit (Universal Language Model Fine-tuning). L' [article] (https://arxiv.org/abs/1801.06146) a montré que cette étape supplémentaire de mise au point du modèle de langage, avant le transfert de l'apprentissage à une tâche de classification, aboutissait à des prédictions nettement meilleures. En utilisant cette approche, nous avons trois étapes pour l'apprentissage par transfert en NLP, comme résumé dans <<ulmfit_process>>.

<img alt="Diagram of the ULMFiT process" width="700" caption="The ULMFiT process" id="ulmfit_process" src="images/att_00027.png">

We'll now explore how to apply a neural network to this language modeling problem, using the concepts introduced in the last two chapters. But before reading further, pause and think about how *you* would approach this.

## Text Preprocessing

La manière dont nous allons utiliser ce que nous avons appris jusqu'à présent pour construire un modèle de langage n'est pas du tout évidente. Les phrases peuvent être de différentes longueurs et les documents peuvent être très longs. Alors, comment pouvons-nous prédire le prochain mot d'une phrase en utilisant un réseau de neurones? Découvrons-le!

Nous avons déjà vu comment les variables catégorielles peuvent être utilisées comme variables indépendantes pour un réseau de neurones. L'approche que nous avons adoptée pour une seule variable catégorielle était de:

1. Faites une liste de tous les niveaux possibles de cette variable catégorielle (nous appellerons cette liste le * vocab *).
1. Remplacez chaque niveau par son index dans le vocabulaire.
1. Créez une matrice d'embeddings (vecteurs représentant les mots) pour cela contenant une ligne pour chaque niveau (c'est-à-dire pour chaque élément du vocabulaire).
1. Utilisez cette matrice d'embeddings comme première couche d'un réseau neuronal. (Une matrice d'embeddings dédiée peut prendre comme entrées les index de vocabulaire bruts créés à l'étape 2; c'est équivalent mais plus rapide et plus efficace qu'une matrice qui prend en entrée des vecteurs encodés en one-hot  représentant les index.)

On peut faire à peu près la même chose avec du texte! Ce qui est nouveau, c'est l'idée d'une séquence. Tout d'abord, nous concaténons tous les documents de notre ensemble de données en une grande et longue chaîne et nous la divisons en mots, ce qui nous donne une très longue liste de mots (ou «tokens»). Notre variable indépendante sera la séquence de mots commençant par le premier mot de notre très longue liste et se terminant par l'avant-dernier, et notre variable dépendante sera la séquence de mots commençant par le deuxième mot et se terminant par le dernier mot.

Notre vocabulaire sera constitué d'un mélange de mots communs qui sont déjà dans le vocabulaire de notre modèle pré-entraîné et de nouveaux mots spécifiques à notre corpus (termes cinématographiques ou noms d'acteurs, par exemple). Notre matrice d'embeddings sera construite en conséquence: pour les mots qui sont dans le vocabulaire de notre modèle pré-entraîné, nous prendrons la ligne correspondante dans la matrice d'inclusion du modèle pré-entraîné; mais pour les nouveaux mots, nous n'aurons rien, donc nous allons simplement initialiser la ligne correspondante avec un vecteur aléatoire.

Chacune des étapes nécessaires pour créer un modèle de langage est associée à un jargon issu du monde du traitement du langage naturel et des classes fastai et PyTorch disponibles pour vous aider. Les étapes sont:

- Tokenisation :: Convertissez le texte en une liste de mots (ou caractères, ou sous-chaînes, selon la granularité de votre modèle)
- Numérisation :: Faites une liste de tous les mots uniques qui apparaissent (le vocabulaire), et convertissez chaque mot en un nombre, en recherchant son index dans le vocabulaire
- Création du chargeur de données de modèle de langage :: fastai fournit une classe `LMDataLoader` qui gère automatiquement la création d'une variable dépendante qui est décalée par rapport à la variable indépendante d'un token. Il gère également certains détails importants, tels que la manière de mélanger les données d'apprentissage de manière à ce que les variables dépendantes et indépendantes conservent leur structure selon les besoins.
- Création de modèle de langage :: Nous avons besoin d'un type spécial de modèle qui fait quelque chose que nous n'avons jamais vu auparavant: gère les listes d'entrée qui peuvent être arbitrairement grandes ou petites. Il y a un certain nombre de façons de le faire; dans ce chapitre, nous utiliserons un * réseau neuronal récurrent * (RNN). Nous aborderons les détails de ces RNN dans le <<chapter_nlp_dive>>, mais pour l'instant, vous pouvez le considérer comme juste un autre réseau neuronal profond.

Voyons comment chaque étape fonctionne en détail.

### Tokenization

Lorsque nous avons dit «convertir le texte en une liste de mots», nous avons omis beaucoup de détails. Par exemple, que faisons-nous de la ponctuation? Comment traitons-nous un mot comme " don't " ? Est-ce un mot ou deux? Qu'en est-il de longs mots médicaux ou chimiques? Devraient-ils être divisés en leurs parties distinctes de signification? Qu'en est-il des mots avec trait d'union? Qu'en est-il des langues comme l'allemand et le polonais où nous pouvons créer de très longs mots à partir de très nombreuses pièces? Qu'en est-il des langues comme le japonais et le chinois qui n'utilisent pas du tout de bases et qui n'ont pas vraiment une idée bien définie de *mot*?

Parce qu'il n'y a pas de réponse correcte à ces questions, il n'y a pas une seule approche de la tokenisation. Il existe trois approches principales:

- Basé sur des mots: divise une phrase sur des espaces, ainsi que l'application de règles spécifiques à la langue pour essayer de séparer les parties du sens même s'il n'y a pas d'espaces (comme transformer «don't» en «do n't») . En règle générale, les signes de ponctuation sont également divisés en tokens distincts.
- Basée sur les sous-mots :: Divise les mots en parties plus petites, en fonction des sous-chaînes les plus courantes. Par exemple, «occasion» peut être symbolisé par «o c ca sion».
- Basé sur les caractères: divise une phrase en ses caractères individuels.

Nous allons examiner la tokenisation des mots et des sous-mots ici, et nous laisserons la tokenisation basée sur les caractères à implémenter dans le questionnaire à la fin de ce chapitre.

> jargon: token: Un élément d'une liste créé par le processus de tokenisation. Il peut s'agir d'un mot, d'une partie d'un mot (un _sous-mot_) ou d'un seul caractère.

### Tokenisation de mots avec fastai

Plutôt que de fournir ses propres tokenizers, fastai fournit à la place une interface cohérente à une gamme de tokenizers dans des bibliothèques externes. La tokenisation est un domaine de recherche actif, et de nouveaux tokenizers améliorés sortent tout le temps, de sorte que les valeurs par défaut utilisées par fastai changent également. Cependant, l'API et les options ne devraient pas trop changer, car fastai essaie de maintenir une API cohérente même lorsque la technologie sous-jacente change.

Essayons-le avec l'ensemble de données IMDb:

In [10]:
from fastai.text.all import *
path = untar_data(URLs.IMDB)

A new version of this dataset is available, downloading...


Nous devrons récupérer les fichiers texte pour essayer un tokenizer. Tout comme `get_image_files`, que nous avons déjà utilisé plusieurs fois, récupère tous les fichiers image dans un chemin,` get_text_files` récupère tous les fichiers texte dans un chemin. Nous pouvons également éventuellement passer des `dossiers` pour restreindre la recherche à une liste particulière de sous-dossiers:

In [11]:
files = get_text_files(path, folders = ['train', 'test', 'unsup'])

Voici une critique que nous allons tokeniser (nous allons simplement en afficher le début ici pour économiser de l'espace):

In [13]:
txt = files[0].open().read(); txt[:75]

"That's right. Ohwon (the painter and the main character) is an exceptional "

Au moment où nous écrivons ce livre, le tokenizer de mot anglais par défaut pour fastai utilise une bibliothèque appelée *spaCy*. Il dispose d'un moteur de règles sophistiqué avec des règles spéciales pour les URL, des mots anglais spéciaux individuels et bien plus encore. Plutôt que d'utiliser directement `SpacyTokenizer`, cependant, nous utiliserons` WordTokenizer`, car cela pointera toujours vers le tokenizer de mot par défaut actuel de fastai (qui peut ne pas être nécessairement spaCy, selon le moment où vous lisez ceci).

Essayons-le. Nous utiliserons la fonction `coll_repr (collection, n)` de fastai pour afficher les résultats. Ceci affiche les premiers *`n`* éléments de *`collection`*, ainsi que la taille complète - c'est ce que `L` utilise par défaut. Notez que les tokeniseurs de fastai prennent une collection de documents à tokeniser, donc nous devons envelopper `txt` dans une liste:

In [14]:
spacy = WordTokenizer()
toks = first(spacy([txt]))
print(coll_repr(toks, 30))

(#126) ['That',"'s",'right','.','Ohwon','(','the','painter','and','the','main','character',')','is','an','exceptional','person','.','What','strikes','me','most','is','the','message','this','film','might','address','to'...]


Comme vous le voyez, spaCy vient principalement de séparer les mots et la ponctuation. Mais il fait autre chose ici aussi: il a divisé «That's» en «That» et «'s». Cela a un sens intuitif; ce sont des mots séparés, vraiment. La tokenisation est une tâche étonnamment subtile, lorsque vous pensez à tous les petits détails qui doivent être traités. Heureusement, spaCy les gère assez bien pour nous - par exemple, nous voyons ici que "." est séparé lorsqu'il termine une phrase, mais pas par un acronyme ou un nombre:

In [15]:
first(spacy(['The U.S. dollar $1 is $1.00.']))

(#9) ['The','U.S.','dollar','$','1','is','$','1.00','.']

fastai ajoute ensuite des fonctionnalités supplémentaires au processus de tokenisation avec la classe `Tokenizer`:

In [16]:
tkn = Tokenizer(spacy)
print(coll_repr(tkn(txt), 31))

(#138) ['xxbos','xxmaj','that',"'s",'right','.','xxmaj','ohwon','(','the','painter','and','the','main','character',')','is','an','exceptional','person','.','xxmaj','what','strikes','me','most','is','the','message','this','film'...]


Notez qu'il y a maintenant des tokens qui commencent par les caractères "xx", qui n'est pas un préfixe de mot courant en anglais. Ce sont des *tokens spéciaux*.

Par exemple, le premier élément de la liste, `xxbos`, est un token spécial qui indique le début d'un nouveau texte (" BOS "est un acronyme NLP standard qui signifie " Beginning Of Stream - début du flux "). En reconnaissant ce token de départ, le modèle pourra apprendre qu'il doit «oublier» ce qui a été dit précédemment et se concentrer sur les mots à venir.

Ces tokens spéciaux ne proviennent pas directement de spaCy. Ils sont là car fastai les ajoute par défaut, en appliquant un certain nombre de règles lors du traitement du texte. Ces règles sont conçues pour permettre à un modèle de reconnaître plus facilement les parties importantes d'une phrase. Dans un sens, nous traduisons la séquence originale de la langue anglaise en une langue simplifiée à tokens - une langue conçue pour être facile à apprendre pour un modèle.

Par exemple, les règles remplaceront une séquence de quatre points d'exclamation par un jeton spécial *caractère répété*, suivi du numéro quatre, puis d'un seul point d'exclamation. De cette manière, la matrice d'embeddings du modèle peut coder des informations sur des concepts généraux tels que la ponctuation répétée plutôt que d'exiger un token distinct pour chaque nombre de répétitions de chaque signe de ponctuation. De même, un mot en majuscule sera remplacé par un token de capitalisation spécial, suivi de la version minuscule du mot. De cette façon, la matrice d'embeddings n'a besoin que des versions minuscules des mots, ce qui économise les ressources de calcul et de mémoire, mais peut toujours apprendre le concept de capitalisation.

Voici quelques-uns des principaux tokens spéciaux que vous verrez:

- `xxbos` :: Indique le début d'un texte (ici, une critique)
- `xxmaj` :: Indique que le mot suivant commence par une majuscule (puisque nous avons tout mis en minuscules)
- `xxunk` :: Indique que le mot suivant est inconnu

Pour voir les règles qui ont été utilisées, vous pouvez vérifier les règles par défaut:

In [17]:
defaults.text_proc_rules

[<function fastai.text.core.fix_html(x)>,
 <function fastai.text.core.replace_rep(t)>,
 <function fastai.text.core.replace_wrep(t)>,
 <function fastai.text.core.spec_add_spaces(t)>,
 <function fastai.text.core.rm_useless_spaces(t)>,
 <function fastai.text.core.replace_all_caps(t)>,
 <function fastai.text.core.replace_maj(t)>,
 <function fastai.text.core.lowercase(t, add_bos=True, add_eos=False)>]

Comme toujours, vous pouvez consulter le code source de chacun d'eux dans un notebook en tapant:

''
?? replace_rep
''

Voici un bref résumé de ce que chacun fait:

- `fix_html` :: Remplace les caractères HTML spéciaux par une version lisible (les critiques IMDb en ont plusieurs)
- `replace_rep` :: Remplace tout caractère répété trois fois ou plus par un token spécial pour la répétition (` xxrep`), le nombre de fois qu'il est répété, puis le caractère
- `replace_wrep` :: Remplace tout mot répété trois fois ou plus par un token spécial pour la répétition de mot (` xxwrep`), le nombre de fois qu'il est répété, puis le mot
- `spec_add_spaces` :: Ajoute des espaces autour de / et #
- `rm_useless_spaces` :: Supprime toutes les répétitions du caractère espace
- `replace_all_caps` :: Rends minuscule un mot écrit en majuscules et ajoute un token spécial pour toutes les majuscules (` xxup`) devant lui
- `replace_maj` :: Rends minuscule un mot en majuscule et ajoute un token spécial de majuscule (` xxmaj`) devant lui
- `lowercase` :: Rends minuscule tout le texte et ajoute un token spécial au début (` xxbos`) et / ou à la fin (`xxeos`)

Jetons un coup d'œil à quelques-uns d'entre eux en action:

In [18]:
coll_repr(tkn('&copy;   Fast.ai www.fast.ai/INDEX'), 31)

"(#11) ['xxbos','©','xxmaj','fast.ai','xxrep','3','w','.fast.ai','/','xxup','index']"

Voyons maintenant comment la tokenisation des sous-mots fonctionnerait.

### Tokenisation des sous-mots

En plus de l'approche de *tokenisation de mot* vue dans la dernière section, une autre méthode de tokenisation populaire est *tokenization de sous-mot*. La tokenisation des mots repose sur l'hypothèse que les espaces fournissent une séparation utile des composants de sens dans une phrase. Cependant, cette hypothèse n'est pas toujours appropriée. Par exemple, considérez cette phrase: 我 的 名字 是 郝杰瑞 ("Je m'appelle Jeremy Howard" en chinois). Cela ne fonctionnera pas très bien avec un tokeniseur de mots, car il n'y a pas d'espace! Des langues comme le chinois et le japonais n'utilisent pas d'espaces, et en fait, elles n'ont même pas un concept bien défini de «mot». Il existe également des langues, comme le turc et le hongrois, qui peuvent ajouter de nombreux sous-mots ensemble sans espaces, créant des mots très longs contenant de nombreuses informations distinctes.

Pour gérer ces cas, il est généralement préférable d'utiliser la tokenisation des sous-mots. Cela se déroule en deux étapes:

1. Analysez un corpus de documents pour trouver les groupes de lettres les plus courants. Ceux-ci deviennent le vocabulaire.
2. Tokeniser le corpus en utilisant ce vocabulaire *d'unités de sous-mots*.

Regardons un exemple. Pour notre corpus, nous utiliserons les 2000 premières critiques de films:

In [19]:
txts = L(o.open().read() for o in files[:2000])

Nous instancions notre tokeniseur, en transmettant la taille du vocabulaire que nous voulons créer, puis nous devons l' «entraîner». Autrement dit, nous devons lui faire lire nos documents et trouver les séquences communes de caractères pour créer le vocabulaire. Ceci est fait avec `setup`. Comme nous le verrons bientôt, `setup` est une méthode fastai spéciale qui est appelée automatiquement dans nos pipelines de traitement de données habituels. Puisque nous faisons tout manuellement pour le moment, cependant, nous devons l'appeler nous-mêmes. Voici une fonction qui effectue ces étapes pour une taille de vocabulaire donnée et montre un exemple de sortie:

In [20]:
def subword(sz):
    sp = SubwordTokenizer(vocab_sz=sz)
    sp.setup(txts)
    return ' '.join(first(sp([txt]))[:40])

Let's try it out:

In [21]:
subword(1000)

"▁Th at ' s ▁right . ▁O h w on ▁( the ▁pa in ter ▁and ▁the ▁main ▁character ) ▁is ▁an ▁exc ept ion al ▁person . ▁W ha t ▁st ri ke s ▁me ▁most ▁is ▁the ▁me"

Lors de l'utilisation du token de sous-mot de fastai, le caractère spécial `▁`  représente un caractère d'espace dans le texte d'origine.

Si nous utilisons un vocabulaire plus petit, chaque token représentera moins de caractères et il en faudra plus pour représenter une phrase:

In [22]:
subword(200)

"▁ T h at ' s ▁ ri g h t . ▁ O h w on ▁ ( th e ▁p a in ter ▁and ▁the ▁ma in ▁ ch ar a c ter ) ▁is ▁ an ▁"

D'un autre côté, si nous utilisons un vocabulaire plus large, les mots anglais les plus courants finiront dans le vocabulaire eux-mêmes, et nous n'en aurons pas besoin d'autant pour représenter une phrase:

In [23]:
subword(10000)

"▁That ' s ▁right . ▁Ohwon ▁( the ▁painter ▁and ▁the ▁main ▁character ) ▁is ▁an ▁exceptional ▁person . ▁What ▁strike s ▁me ▁most ▁is ▁the ▁message ▁this ▁film ▁might ▁address ▁to ▁all ▁of ▁you ▁people ▁there . ▁And ▁the"

Choisir la taille du vocabulaire d'un sous-mot représente un compromis: un vocabulaire plus grand signifie moins de token par phrase, ce qui signifie un entraînement plus rapide, moins de mémoire et moins d'état à retenir pour le modèle; mais en revanche, cela signifie des matrices d'intégration plus grandes, qui nécessitent plus de données pour apprendre.

Dans l'ensemble, la tokenisation de sous-mots fournit un moyen d'évoluer facilement entre la tokenisation de caractères (c'est-à-dire en utilisant un petit vocabulaire de sous-mots) et la tokenisation de mots (c'est-à-dire en utilisant un grand vocabulaire de sous-mots), et gère chaque langue humaine sans avoir besoin de développer des algorithmes spécifiques à la langue. Il peut même gérer d'autres "langages" tels que les séquences génomiques ou la notation musicale MIDI! Pour cette raison, au cours de la dernière année, sa popularité a grimpé en flèche et il semble probable qu'elle devienne l'approche de tokenisation la plus courante (il se peut qu'elle le soit déjà, au moment où vous lirez ceci!).

Une fois que nos textes ont été divisés en tokens, nous devons les convertir en nombres. Nous examinerons cela ensuite.

### Numérisation avec fastai

*La numérisation* est le processus de mappage de tokens en nombres entiers. Les étapes sont fondamentalement identiques à celles nécessaires pour créer une variable `Category`, telle que la variable dépendante des chiffres dans MNIST:

1. Faites une liste de tous les niveaux possibles de cette variable catégorielle (le vocabulaire).
1. Remplacez chaque niveau par son index dans le vocabulaire.

Jetons un coup d'œil à cela en action sur le texte à jeton de mot que nous avons vu plus tôt:

In [24]:
toks = tkn(txt)
print(coll_repr(tkn(txt), 31))

(#138) ['xxbos','xxmaj','that',"'s",'right','.','xxmaj','ohwon','(','the','painter','and','the','main','character',')','is','an','exceptional','person','.','xxmaj','what','strikes','me','most','is','the','message','this','film'...]


Tout comme avec `SubwordTokenizer`, nous devons appeler` setup` sur `Numericalize`; c'est ainsi que nous créons le vocabulaire. Cela signifie que nous aurons d'abord besoin de notre corpus tokenisé. Puisque la tokenisation prend un certain temps, elle est effectuée en parallèle par fastai; mais pour cette procédure pas à pas manuelle, nous utiliserons un petit sous-ensemble:

In [25]:
toks200 = txts[:200].map(tkn)
toks200[0]

(#138) ['xxbos','xxmaj','that',"'s",'right','.','xxmaj','ohwon','(','the'...]

Nous pouvons les passer à `setup` créer notre vocabulaire:

In [26]:
num = Numericalize()
num.setup(toks200)
coll_repr(num.vocab,20)

"(#2136) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','the','.',',','and','a','of','to','is','in','it','i'...]"

Nos tokens de règles spéciales apparaissent en premier, puis chaque mot apparaît une fois, par ordre de fréquence. Les valeurs par défaut de `Numericalize` sont` min_freq = 3, max_vocab = 60000`. `max_vocab = 60000` entraîne le remplacement par fastai de tous les mots autres que les 60000 les plus courants par un jeton spécial *mot inconnu*,` xxunk`. Ceci est utile pour éviter d'avoir une matrice d'embedding trop volumineuse, car cela peut ralentir l'entraînement et utiliser trop de mémoire, et peut également signifier qu'il n'y a pas assez de données pour apprendre des représentations utiles pour des mots rares. Cependant, ce dernier problème est mieux géré en définissant `min_freq`; la valeur par défaut «min_freq = 3» signifie que tout mot apparaissant moins de trois fois est remplacé par «xxunk».

fastai peut également numériser votre jeu de données en utilisant un vocabulaire que vous fournissez, en passant une liste de mots comme paramètre «vocab».

Une fois que nous avons créé notre objet `Numericalize`, nous pouvons l'utiliser comme s'il s'agissait d'une fonction:

In [27]:
nums = num(toks)[:20]; nums

TensorText([  2,   8,  20,  21, 276,  10,   8,   0,  37,   9,   0,  12,   9, 235, 126,  36,  16,  53,   0, 321])

Cette fois, nos tokens ont été convertis en un tenseur d'entiers que notre modèle peut recevoir. Nous pouvons vérifier qu'ils correspondent au texte d'origine:

In [28]:
' '.join(num.vocab[o] for o in nums)

"xxbos xxmaj that 's right . xxmaj xxunk ( the xxunk and the main character ) is an xxunk person"

Maintenant que nous avons des nombres, nous devons les mettre en lots pour notre modèle.

### Mettre nos textes en batches (lots) pour un modèle de langage

Lorsqu'il s'agissait d'images, nous devions les redimensionner toutes à la même hauteur et largeur avant de les regrouper dans un mini-lot afin qu'elles puissent s'empiler efficacement dans un seul tenseur. Ici, ce sera un peu différent, car on ne peut pas simplement redimensionner le texte à la longueur souhaitée. De plus, nous voulons que notre modèle de langage lise le texte dans l'ordre, afin qu'il puisse prédire efficacement quel est le mot suivant. Cela signifie que chaque nouveau lot doit commencer exactement là où le précédent s'est arrêté.

Supposons que nous ayons le texte suivant:

>: Dans ce chapitre, nous reviendrons sur l'exemple de classification des critiques de films que nous avons étudiées au chapitre 1 et creuserons plus profondément sous la surface. Nous examinerons d'abord les étapes de traitement nécessaires pour convertir du texte en nombres et comment le personnaliser. En faisant cela, nous aurons un autre exemple du préprocesseur utilisé dans l'API du bloc de données. \nPuis nous étudierons comment nous construisons un modèle de langage et le l'entraînons pendant un certain temps.

Le processus de tokenisation ajoutera des tokens spéciaux et traitera la ponctuation pour renvoyer ce texte:

>: xxbos xxmaj dans ce chapitre, nous reviendrons sur l'exemple de classification des critiques de films que nous avons étudiées au chapitre 1 et creuserons plus profondément sous la surface. xxmaj nous examinerons d'abord les étapes de traitement nécessaires pour convertir du texte en nombres et comment le personnaliser. xxmaj en faisant cela, nous aurons un autre exemple du préprocesseur utilisé dans le bloc de données xxup api. \ n xxmaj alors nous étudierons comment nous construisons un modèle de langage et le formons pendant un certain temps.

Nous avons maintenant 90 tokens séparés par des espaces. Disons que nous voulons une taille de batch de 6. Nous devons diviser ce texte en 6 parties contiguës de longueur 15:

In [29]:
#hide_input
stream = "In this chapter, we will go back over the example of classifying movie reviews we studied in chapter 1 and dig deeper under the surface. First we will look at the processing steps necessary to convert text into numbers and how to customize it. By doing this, we'll have another example of the PreProcessor used in the data block API.\nThen we will study how we build a language model and train it for a while."
tokens = tkn(stream)
bs,seq_len = 6,15
d_tokens = np.array([tokens[i*seq_len:(i+1)*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

xxbos,xxmaj,in,this,chapter,",",we,will,go,back,over,the,example,of,classifying
movie,reviews,we,studied,in,chapter,1,and,dig,deeper,under,the,surface,.,xxmaj
first,we,will,look,at,the,processing,steps,necessary,to,convert,text,into,numbers,and
how,to,customize,it,.,xxmaj,by,doing,this,",",we,'ll,have,another,example
of,the,preprocessor,used,in,the,data,block,xxup,api,.,\n,xxmaj,then,we
will,study,how,we,build,a,language,model,and,train,it,for,a,while,.


Dans un monde parfait, nous pourrions alors donner ce lot à notre modèle. Mais cette approche ne s'adapte pas, car en dehors de cet exemple simpliste, il est peu probable qu'un seul lot contenant tous les textes tienne dans la mémoire de notre GPU (ici nous avons 90 tokens mais toutes les critiques IMDb ensemble donnent plusieurs millions).

Nous devons donc diviser ce tableau plus finement en sous-tableaux d'une longueur de séquence fixe. Il est important de maintenir l'ordre dans et entre ces sous-tableaux, car nous utiliserons un modèle qui maintient un état afin qu'il se souvienne de ce qu'il a lu précédemment lors de la prédiction de la suite.

Pour revenir à notre exemple précédent avec 6 lots de longueur 15, si nous choisissons une longueur de séquence de 5, cela voudrait dire que nous alimentons d'abord le tableau suivant:

In [30]:
#hide_input
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15:i*15+seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

xxbos,xxmaj,in,this,chapter
movie,reviews,we,studied,in
first,we,will,look,at
how,to,customize,it,.
of,the,preprocessor,used,in
will,study,how,we,build


Puis celui-ci:

In [31]:
#hide_input
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15+seq_len:i*15+2*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

",",we,will,go,back
chapter,1,and,dig,deeper
the,processing,steps,necessary,to
xxmaj,by,doing,this,","
the,data,block,xxup,api
a,language,model,and,train


Et finalement:

In [32]:
#hide_input
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15+10:i*15+15] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

over,the,example,of,classifying
under,the,surface,.,xxmaj
convert,text,into,numbers,and
we,'ll,have,another,example
.,\n,xxmaj,then,we
it,for,a,while,.


Pour revenir à notre ensemble de données sur les critiques de films, la première étape consiste à transformer les textes individuels en un flux en les concaténant ensemble. Comme pour les images, il est préférable de randomiser l'ordre des entrées, donc au début de chaque époque, nous mélangerons les entrées pour créer un nouveau flux (nous mélangeons l'ordre des documents, pas l'ordre des mots à l'intérieur, ou les textes n'auraient plus de sens!).

Nous avons ensuite découpé ce flux en un certain nombre de lots (qui est notre *taille de lot*). Par exemple, si le flux a 50000 tokens et que nous définissons une taille de lot de 10, cela nous donnera 10 mini-flux de 5000 tokens. L'important est de conserver l'ordre des tokens (donc de 1 à 5000 pour le premier mini-flux, puis de 5001 à 10000 ...), car nous voulons que le modèle lise des lignes de texte continues (comme dans le exemple précédent). Un jeton `xxbos` est ajouté au début de chacun pendant le prétraitement, afin que le modèle sache quand il lit le flux quand une nouvelle entrée commence.

Donc, pour récapituler, à chaque époque, nous mélangeons notre collection de documents et les concaténons dans un flux de tokens. Nous avons ensuite coupé ce flux en un lot de mini-flux consécutifs de taille fixe. Notre modèle lira alors les mini-flux dans l'ordre, et grâce à un état interne, il produira la même activation quelle que soit la longueur de séquence que nous avons choisie.

Tout cela est fait dans les coulisses par la bibliothèque fastai lorsque nous créons un `LMDataLoader`. Nous faisons cela en appliquant d'abord notre objet `Numericalize` aux textes tokenisés:

In [33]:
nums200 = toks200.map(num)

puis en le passant au `LMDataLoader`:

In [34]:
dl = LMDataLoader(nums200)

Confirmons que cela donne les résultats attendus en tirant le premier lot:

In [35]:
x,y = first(dl)
x.shape,y.shape

(torch.Size([64, 72]), torch.Size([64, 72]))

puis en regardant la première ligne de la variable indépendante, qui devrait être le début du premier texte:

In [36]:
' '.join(num.vocab[o] for o in x[0][:20])

"xxbos xxmaj that 's right . xxmaj xxunk ( the xxunk and the main character ) is an xxunk person"

La variable dépendante est la même chose décalée d'un token:

In [37]:
' '.join(num.vocab[o] for o in y[0][:20])

"xxmaj that 's right . xxmaj xxunk ( the xxunk and the main character ) is an xxunk person ."

Ceci conclut toutes les étapes de prétraitement que nous devons appliquer à nos données. Nous sommes maintenant prêts à entraîner notre classificateur de texte.

## Entraînement d'un Classifieur de Texte

Comme nous l'avons vu au début de ce chapitre, il y a deux étapes pour former un classifieur de texte à la pointe de la technologie à l'aide de l'apprentissage par transfert: nous devons d'abord affiner notre modèle de langage pré-entraîné sur Wikipédia au corpus de critiques IMDb, puis nous pouvons utiliser ce modèle pour former un classifieur.

Comme d'habitude, commençons par assembler nos données.

###  Modèle de Langue utilisant l'API DataBlock

fastai gère automatiquement la tokenisation et la numérisation lorsque `TextBlock` est passé à` DataBlock`. Tous les arguments qui peuvent être passés à `Tokenize` et` Numericalize` peuvent également être passés à `TextBlock`. Dans le chapitre suivant, nous discuterons des moyens les plus simples d'exécuter chacune de ces étapes séparément, pour faciliter le débogage, mais vous pouvez toujours simplement les déboguer en les exécutant manuellement sur un sous-ensemble de vos données, comme indiqué dans les sections précédentes. Et n'oubliez pas la méthode pratique `summary` de` DataBlock`, qui est très utile pour déboguer les problèmes de données.

Voici comment nous utilisons `TextBlock` pour créer un modèle de langage, en utilisant les valeurs par défaut de fastai:

In [38]:
get_imdb = partial(get_text_files, folders=['train', 'test', 'unsup'])

dls_lm = DataBlock(
    blocks=TextBlock.from_folder(path, is_lm=True),
    get_items=get_imdb, splitter=RandomSplitter(0.1)
).dataloaders(path, path=path, bs=128, seq_len=80)

Une chose qui diffère des types précédents que nous avons utilisés dans `DataBlock` est que nous n'utilisons pas simplement la classe directement (c'est-à-dire` TextBlock (...) `, mais appelons à la place une *méthode de classe*.  Une méthode de classe est une méthode Python qui, comme son nom l'indique, appartient à une *classe* plutôt qu'à un *objet*. (Assurez-vous de rechercher en ligne plus d'informations sur les méthodes de classe si vous ne les connaissez pas, car elles sont couramment utilisées dans de nombreuses bibliothèques et applications Python; nous les avons utilisés quelques fois auparavant dans le livre, mais nous n'avons pas attiré l'attention sur elles.) La raison pour laquelle `TextBlock` est spécial est que la configuration du vocabulaire du numériseur peut prendre un longtemps (nous devons lire et tokeniser chaque document pour obtenir le vocabulaire). Pour être aussi efficace que possible, il effectue quelques optimisations:

- Il enregistre les documents tokenisés dans un dossier temporaire, il n'est donc pas nécessaire de les tokeniser plus d'une fois
- Il exécute plusieurs processus de tokenisation en parallèle, pour tirer parti des processeurs de votre ordinateur

Nous devons dire à `TextBlock` comment accéder aux textes, afin qu'il puisse faire ce prétraitement initial - c'est ce que fait` from_folder`.

`show_batch` fonctionne alors de la manière habituelle:

In [39]:
dls_lm.show_batch(max_n=2)

,text,text_
0,"xxbos xxmaj the thing viewers will remember most is the bad headache the movie has given them due to the overly flashy , shaky , camera - work and the fast , confusing cutting . i am not against those kind of stylistic devices if they are done right like xxmaj oliver xxmaj stone and xxmaj steven xxmaj soderbergh proof with most of their movies , but in this case there was xxup way too much . xxmaj it seems","xxmaj the thing viewers will remember most is the bad headache the movie has given them due to the overly flashy , shaky , camera - work and the fast , confusing cutting . i am not against those kind of stylistic devices if they are done right like xxmaj oliver xxmaj stone and xxmaj steven xxmaj soderbergh proof with most of their movies , but in this case there was xxup way too much . xxmaj it seems like"
1,"of bullets into the bad guy alone ) . \n\n xxmaj familiar scenes are replayed again by people now so rich ( and yes , xxmaj i 'm thinking xxmaj eddie xxmaj murphy here in a redneck bar ) that the good guys come off as cruel rather than heroic . \n\n xxmaj xxunk xxmaj walter can still trade gunfire with anyone out there , and for that reason alone it 's worthy of your attention . xxmaj it also","bullets into the bad guy alone ) . \n\n xxmaj familiar scenes are replayed again by people now so rich ( and yes , xxmaj i 'm thinking xxmaj eddie xxmaj murphy here in a redneck bar ) that the good guys come off as cruel rather than heroic . \n\n xxmaj xxunk xxmaj walter can still trade gunfire with anyone out there , and for that reason alone it 's worthy of your attention . xxmaj it also looks"


À présent nos données sont prêtes, nous pouvons affiner (fine-tune) le modèle de langage pré-entraîné.

### Affiner le modèle de langage

Pour convertir les indices de mots entiers en activations que nous pouvons utiliser pour notre réseau de neurones, nous utiliserons des embeddings, tout comme nous l'avons fait pour le filtrage collaboratif et la modélisation tabulaire. Ensuite, nous allons alimenter ces embeddings dans un *réseau neuronal récurrent* (RNN), en utilisant une architecture appelée * AWD-LSTM * (nous allons vous montrer comment écrire un tel modèle à partir de zéro dans <<chapter_nlp_dive>>). Comme nous l'avons vu précédemment, les embeddings dans le modèle pré-entraîné sont fusionnées avec des embeddings aléatoires ajoutées pour les mots qui ne faisaient pas partie du vocabulaire de pré-apprentissage. Ceci est géré automatiquement dans `language_model_learner`:

In [41]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3, 
    metrics=[accuracy, Perplexity()]).to_fp16()

/home/ismael/Utils/miniconda2/envs/fastai2/lib/python3.8/site-packages/torch/cuda/amp/autocast_mode.py:114: UserWarning: torch.cuda.amp.autocast only affects CUDA ops, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.autocast only affects CUDA ops, but CUDA is not available.  Disabling.")


La fonction de perte utilisée par défaut est la perte d'entropie croisée, car nous avons essentiellement un problème de classification (les différentes catégories étant les mots de notre vocabulaire). La métrique *perplexité* utilisée ici est souvent utilisée en NLP pour les modèles de langage: c'est l'exponentielle de la perte (c'est-à-dire `torch.exp (cross_entropy)`). Nous incluons également la métrique de précision, pour voir combien de fois notre modèle a raison lorsqu'il tente de prédire le mot suivant, car l'entropie croisée (comme nous l'avons vu) est à la fois difficile à interpréter et nous en dit plus sur la confiance du modèle que sa précision.

Revenons au diagramme de processus du début de ce chapitre. La première flèche a été complétée pour nous et rendue disponible en tant que modèle pré-entraîné dans fastai, et nous venons de construire les `DataLoaders` et` Learner` pour la deuxième étape. Nous sommes maintenant prêts à affiner notre modèle de langage!


<img alt="Diagram of the ULMFiT process" width="450" src="images/att_00027.png">

L'entraînement à chaque époque prend un certain temps, nous enregistrerons donc les résultats du modèle intermédiaire pendant le processus d'entraînement. Puisque `fine_tune` ne fait pas cela pour nous, nous utiliserons` fit_one_cycle`. Tout comme `cnn_learner`,` language_model_learner` appelle automatiquement `freeze` lors de l'utilisation d'un modèle pré-entraîné (qui est la valeur par défaut), donc cela entraînera uniquement les plongements (la seule partie du modèle qui contient des pondérations initialisées aléatoirement, c'est-à-dire des plongements pour mots qui sont dans notre vocabulaire IMDb, mais qui ne sont pas dans le vocabulaire du modèle pré-entraîné):

In [ ]:
learn.fit_one_cycle(1, 2e-2)

/home/ismael/Utils/miniconda2/envs/fastai2/lib/python3.8/site-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


epoch,train_loss,valid_loss,accuracy,perplexity,time


Ce modèle prend un certain temps à s'entraîner, c'est donc une bonne occasion de parler de la sauvegarde des résultats intermédiaires.

### Enregistrer et Charger des Modèles

Vous pouvez facilement sauvegarder l'état de votre modèle comme suit:

In [ ]:
learn.save('1epoch')

Cela créera un fichier dans `learn.path /models/` nommé *1epoch.pth*. Si vous souhaitez charger votre modèle sur une autre machine après avoir créé votre `Learner` de la même manière, ou reprendre l'entraînement plus tard, vous pouvez charger le contenu de ce fichier avec:

In [ ]:
learn = learn.load('1epoch')

Une fois la formation initiale terminée, nous pouvons continuer à affiner le modèle après le dégel:

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10, 2e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.893486,3.772820,0.317104,43.502548,12:37
1,3.820479,3.717197,0.323790,41.148880,12:30
2,3.735622,3.659760,0.330321,38.851997,12:09
3,3.677086,3.624794,0.333960,37.516987,12:12
4,3.636646,3.601300,0.337017,36.645859,12:05
5,3.553636,3.584241,0.339355,36.026001,12:04
6,3.507634,3.571892,0.341353,35.583862,12:08
7,3.444101,3.565988,0.342194,35.374371,12:08
8,3.398597,3.566283,0.342647,35.384815,12:11
9,3.375563,3.568166,0.342528,35.451500,12:05


Une fois que cela est fait, nous sauvegardons tout notre modèle à l'exception de la couche finale qui convertit les activations en probabilités de choisir chaque token de notre vocabulaire. Le modèle n'incluant pas la couche finale est appelé *encodeur*. Nous pouvons le sauvegarder avec `save_encoder`:

In [ ]:
learn.save_encoder('finetuned')

> jargon: Encodeur: Le modèle n'inclut pas la ou les couches finales spécifiques à la tâche. Ce terme signifie à peu près la même chose que _body_ lorsqu'il est appliqué aux CNN de la vision, mais "encodeur" a tendance à être plus utilisé pour la NLP et les modèles génératifs.

This completes the second stage of the text classification process: fine-tuning the language model. We can now use it to fine-tune a classifier using the IMDb sentiment labels.

### Text Generation

Avant de passer à la mise au point du classifieur, essayons rapidement quelque chose de différent: utiliser notre modèle pour générer des critiques aléatoires. Puisqu'il est formé pour deviner quel est le mot suivant de la phrase, nous pouvons utiliser le modèle pour écrire de nouvelles critiques:

In [ ]:
TEXT = "I liked this movie because"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

In [ ]:
print("\n".join(preds))

i liked this movie because of its story and characters . The story line was very strong , very good for a sci - fi film . The main character , Alucard , was very well developed and brought the whole story
i liked this movie because i like the idea of the premise of the movie , the ( very ) convenient virus ( which , when you have to kill a few people , the " evil " machine has to be used to protect


Comme vous pouvez le voir, nous ajoutons du caractère aléatoire (nous choisissons un mot aléatoire en fonction des probabilités renvoyées par le modèle) afin de ne pas obtenir exactement le même avis deux fois. Notre modèle n'a aucune connaissance programmée de la structure d'une phrase ou des règles de grammaire, mais il a clairement beaucoup appris sur les phrases anglaises: nous pouvons le voir mettre en majuscule correctement (*I* est juste transformé en *i* parce que nos règles nécessitent deux caractères ou plus pour considérer un mot comme en majuscule, il est donc normal de le voir en minuscules) et utilise un temps cohérent. L'examen général a du sens à première vue, et ce n'est que si vous lisez attentivement que vous pouvez remarquer que quelque chose ne va pas. Pas mal pour un modèle entraîné en quelques heures!

Mais notre objectif final n'était pas de former un modèle pour générer des avis, mais de les classer ... alors utilisons ce modèle pour faire exactement cela.

### Créer le DataLoaders du Classifieur

Nous passons maintenant de la mise au point du modèle de langage à la mise au point du classifieur. Pour récapituler, un modèle de langage prédit le mot suivant d'un document, il n'a donc pas besoin d'étiquettes externes. Un classifieur,  cependant, prédit une étiquette externe - dans le cas d'IMDb, c'est le sentiment d'un document.

Cela signifie que la structure de notre `DataBlock` pour la classification NLP vous semblera très familière. C'est en fait presque le même que celui que nous avons vu pour les nombreux ensembles de données de classification d'images avec lesquels nous avons travaillé:

In [9]:
dls_clas = DataBlock(
    blocks=(TextBlock.from_folder(path, vocab=dls_lm.vocab),CategoryBlock),
    get_y = parent_label,
    get_items=partial(get_text_files, folders=['train', 'test']),
    splitter=GrandparentSplitter(valid_name='test')
).dataloaders(path, path=path, bs=128, seq_len=72)

NameError: name 'path' is not defined

Tout comme avec la classification d'images, `show_batch` affiche la variable dépendante (sentiment, dans ce cas) avec chaque variable indépendante (texte de la critique de film):

In [ ]:
dls_clas.show_batch(max_n=3)

,text,category
0,"xxbos i rate this movie with 3 skulls , only coz the girls knew how to scream , this could 've been a better movie , if actors were better , the twins were xxup ok , i believed they were evil , but the eldest and youngest brother , they sucked really bad , it seemed like they were reading the scripts instead of acting them … . spoiler : if they 're vampire 's why do they freeze the blood ? vampires ca n't drink frozen blood , the sister in the movie says let 's drink her while she is alive … .but then when they 're moving to another house , they take on a cooler they 're frozen blood . end of spoiler \n\n it was a huge waste of time , and that made me mad coz i read all the reviews of how",neg
1,"xxbos i have read all of the xxmaj love xxmaj come xxmaj softly books . xxmaj knowing full well that movies can not use all aspects of the book , but generally they at least have the main point of the book . i was highly disappointed in this movie . xxmaj the only thing that they have in this movie that is in the book is that xxmaj missy 's father comes to xxunk in the book both parents come ) . xxmaj that is all . xxmaj the story line was so twisted and far fetch and yes , sad , from the book , that i just could n't enjoy it . xxmaj even if i did n't read the book it was too sad . i do know that xxmaj pioneer life was rough , but the whole movie was a downer . xxmaj the rating",neg
2,"xxbos xxmaj this , for lack of a better term , movie is lousy . xxmaj where do i start … … \n\n xxmaj cinemaphotography - xxmaj this was , perhaps , the worst xxmaj i 've seen this year . xxmaj it looked like the camera was being tossed from camera man to camera man . xxmaj maybe they only had one camera . xxmaj it gives you the sensation of being a volleyball . \n\n xxmaj there are a bunch of scenes , haphazardly , thrown in with no continuity at all . xxmaj when they did the ' split screen ' , it was absurd . xxmaj everything was squished flat , it looked ridiculous . \n\n xxmaj the color tones were way off . xxmaj these people need to learn how to balance a camera . xxmaj this ' movie ' is poorly made , and",neg


En regardant la définition de `DataBlock`, chaque élément est familier des blocs de données précédents que nous avons construits, à deux exceptions importantes près:

- `TextBlock.from_folder` n'a plus le paramètre` is_lm = True`.
- Nous transmettons le «vocab» que nous avons créé pour la mise au point du modèle de langage.

La raison pour laquelle nous transmettons le «vocab» du modèle de langage est de nous assurer que nous utilisons la même correspondance de token à indexer. Sinon, les embeddings que nous avons apprises dans notre modèle de langage affiné n'auront aucun sens pour ce modèle, et l'étape d'affinage ne sera d'aucune utilité.

En passant `is_lm = False` (ou en ne passant pas du tout` is_lm`, car il vaut par défaut `False`), nous disons à` TextBlock` que nous avons des données étiquetées régulières, plutôt que d'utiliser les tokens suivants comme étiquettes. Il y a cependant un défi auquel nous devons faire face, qui est lié à l'assemblage de plusieurs documents en un mini-lot. Voyons avec un exemple, en essayant de créer un mini-lot contenant les 10 premiers documents. Nous allons d'abord les numériser:

In [ ]:
nums_samp = toks200[:10].map(num)

Voyons maintenant le nombre de tokens de chacune de ces 10 critiques de films:

In [ ]:
nums_samp.map(len)

(#10) [228,238,121,290,196,194,533,124,581,155]

N'oubliez pas que PyTorch `DataLoader`s doit rassembler tous les éléments d'un lot en un seul tenseur, et un seul tenseur a une forme fixe (c'est-à-dire qu'il a une longueur particulière sur chaque axe, et tous les éléments doivent être cohérents). Cela devrait sembler familier: nous avons eu le même problème avec les images. Dans ce cas, nous avons utilisé le recadrage, le remplissage et / ou l'écrasement pour que toutes les entrées aient la même taille. Le recadrage n'est peut-être pas une bonne idée pour les documents, car il semble probable que nous supprimions certaines informations clés (cela dit, le même problème est vrai pour les images, et nous utilisons le recadrage là-bas; l'augmentation des données n'a pas été bien explorée pour le NLP. Pourtant, il existe peut-être aussi des opportunités d'utiliser le recadrage en NLP!). Vous ne pouvez pas vraiment "écraser" un document. Alors ça laisse du rembourrage!

Nous élargirons les textes les plus courts pour les rendre tous de la même taille. Pour ce faire, nous utilisons un token de remplissage spécial qui sera ignoré par notre modèle. De plus, pour éviter les problèmes de mémoire et améliorer les performances, nous regrouperons des textes qui sont à peu près de la même longueur (avec un peu de mélange pour l'ensemble d'entraînement). Nous faisons cela en triant (approximativement, pour l'ensemble d'apprentissage) les documents par longueur avant chaque époque. Le résultat de ceci est que les documents rassemblés en un seul lot auront tendance à être de longueurs similaires. Nous ne remplirons pas chaque lot à la même taille, mais utiliserons à la place la taille du plus grand document de chaque lot comme taille cible. (Il est possible de faire quelque chose de similaire avec des images, ce qui est particulièrement utile pour les images rectangulaires de taille irrégulière, mais au moment de la rédaction de cet article, aucune bibliothèque ne fournit encore un bon support pour cela, et aucun article ne le couvre. C'est quelque chose que nous '' Cependant, si vous prévoyez d'ajouter bientôt à fastai, gardez un œil sur le site Web du livre; nous ajouterons des informations à ce sujet dès que cela fonctionnera bien.)

Le tri et le remplissage sont automatiquement effectués par l'API du bloc de données pour nous lors de l'utilisation d'un `TextBlock`, avec` is_lm = False`. (Nous n'avons pas ce même problème pour les données de modèle de langage, puisque nous concaténons d'abord tous les documents ensemble, puis les divisons en sections de taille égale.)

Nous pouvons maintenant créer un modèle pour classer nos textes:

In [ ]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, 
                                metrics=accuracy).to_fp16()

La dernière étape avant la formation du classifieur consiste à charger l'encodeur à partir de notre modèle de langage affiné. Nous utilisons `load_encoder` au lieu de` load` car nous n'avons que des poids pré-entraînés disponibles pour l'encodeur; `load` par défaut lève une exception si un modèle incomplet est chargé:

In [ ]:
learn = learn.load_encoder('finetuned')

### Affiner le classifieur

La dernière étape consiste à s'entraîner avec des taux d'apprentissage discriminants et *un dégel progressif*. En vision par ordinateur, nous dégelons souvent le modèle en même temps, mais pour les classificateurs NLP, nous constatons que le dégel de quelques couches à la fois fait une réelle différence:

In [ ]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.347427,0.184480,0.929320,00:33


En une seule époque, nous obtenons le même résultat que notre formation en <<chapter_intro>>: pas trop mal! Nous pouvons passer `-2` à` freeze_to` pour tout geler sauf les deux derniers groupes de paramètres:

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.247763,0.171683,0.934640,00:37


Ensuite, nous pouvons dégeler un peu plus et continuer à nous entraîner:

In [ ]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.193377,0.156696,0.941200,00:45


Et finalement le modèle entier!

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.172888,0.153770,0.943120,01:01
1,0.161492,0.155567,0.942640,00:57


Nous avons atteint une précision de 94,3%, ce qui était une performance de pointe il y a à peine trois ans. En entraînant un autre modèle sur tous les textes lus à l'envers et en faisant la moyenne des prédictions de ces deux modèles, nous pouvons même atteindre une précision de 95,1%, ce qui était l'état de l'art introduit par le papier ULMFiT. Il a été battu il y a seulement quelques mois, en peaufinant un modèle beaucoup plus grand et en utilisant des techniques coûteuses d'augmentation des données (traduction de phrases dans une autre langue et inversement, utilisation d'un autre modèle pour la traduction).

L'utilisation d'un modèle pré-entraîné nous a permis de construire un modèle de langage affiné qui était assez puissant, soit pour générer de fausses critiques, soit pour les classer. Ce sont des choses passionnantes, mais il est bon de se rappeler que cette technologie peut également être utilisée à des fins malveillantes.

## Désinformation et Modèles de Langue

Même de simples algorithmes basés sur des règles, avant les jours des modèles de langage d'apprentissage profond largement disponibles, pourraient être utilisés pour créer des comptes frauduleux et tenter d'influencer les décideurs. Jeff Kao, désormais journaliste informatique chez ProPublica, a analysé les commentaires envoyés à la Federal Communications Commission (FCC) des États-Unis concernant une proposition de 2017 visant à abroger la neutralité du net. Dans son article ["Plus d'un million de commentaires pro-abrogation sur la neutralité du Net ont probablement été truqués"](https://hackernoon.com/more-than-a-million-pro-repeal-net-neutrality-comments-were-lusted-faked-e9f0e3ed36a6), il raconte comment il a découvert un grand groupe de commentaires opposés à la neutralité du net qui semblaient avoir été générés par une sorte de publipostage à la Mad Libs. Dans <<disinformation>>, les faux commentaires ont été utilement codés par couleur par Kao pour mettre en évidence leur nature de formule.

<img src="images/ethics/image16.png" width="700" id="disinformation" caption="Comments received by the FCC during the net neutrality debate">

Kao a estimé que «moins de 800 000 commentaires sur plus de 22 millions… pourraient être considérés comme vraiment uniques» et que «plus de 99% des commentaires vraiment uniques étaient en faveur du maintien de la neutralité du Net».

Compte tenu des progrès réalisés dans la modélisation du langage depuis 2017, de telles campagnes frauduleuses pourraient être presque impossibles à détecter maintenant. Vous disposez désormais de tous les outils nécessaires pour créer un modèle de langage convaincant, c'est-à-dire quelque chose qui peut générer un texte crédible et adapté au contexte. Ce ne sera pas nécessairement parfaitement exact ou correct, mais ce sera plausible. Pensez à ce que signifierait cette technologie lorsqu'elle est associée aux types de campagnes de désinformation dont nous avons entendu parler ces dernières années. Jetez un œil au dialogue Reddit montré dans <<ethics_reddit>>, où un modèle de langage basé sur l'algorithme GPT-2 d'OpenAI a une conversation avec lui-même pour savoir si le gouvernement américain devrait réduire les dépenses de défense.

<img src="images/ethics/image14.png" id="ethics_reddit" caption="An algorithm talking to itself on Reddit" alt="An algorithm talking to itself on Reddit" width="600">

Dans ce cas, il a été explicitement dit qu'un algorithme était utilisé, mais imaginez ce qui se passerait si un mauvais acteur décidait de libérer un tel algorithme sur les réseaux sociaux. Ils pouvaient le faire lentement et prudemment, permettant à l'algorithme de développer progressivement des adeptes et de faire confiance au fil du temps. Il ne faudrait pas beaucoup de ressources pour avoir littéralement des millions de comptes pour faire cela. Dans une telle situation, nous pourrions facilement imaginer arriver à un point où la grande majorité du discours en ligne provenait de robots, et personne n'aurait la moindre idée que cela se produisait.

Nous commençons déjà à voir des exemples d'apprentissage automatique utilisés pour générer des identités. Par exemple, <<katie_jones>> affiche un profil LinkedIn pour Katie Jones.

<img src="images/ethics/image15.jpeg" width="400" id="katie_jones" caption="Katie Jones's LinkedIn profile">

Katie Jones a été connectée sur LinkedIn à plusieurs membres des grands groupes de réflexion de Washington. Mais elle n'existait pas. Cette image que vous voyez a été générée automatiquement par un réseau antagoniste génératif, et quelqu'un du nom de Katie Jones n'est pas, en fait, diplômé du Center for Strategic and International Studies.

Beaucoup de gens supposent ou espèrent que les algorithmes viendront à notre défense ici - que nous développerons des algorithmes de classification capables de reconnaître automatiquement le contenu généré automatiquement. Le problème, cependant, est qu'il s'agira toujours d'une course aux armements, dans laquelle de meilleurs algorithmes de classification (ou discriminateurs) peuvent être utilisés pour créer de meilleurs algorithmes de génération

## Conclusion

Dans ce chapitre, nous avons exploré la dernière application couverte par la bibliothèque fastai: text. Nous avons vu deux types de modèles: des modèles de langage qui peuvent générer des textes et un classificateur qui détermine si un avis est positif ou négatif. Pour construire un classifieur à la pointe de la technologie, nous avons utilisé un modèle de langage pré-entraîné, l'a affiné au corpus de notre tâche, puis utilisé son corps (l'encodeur) avec une nouvelle tête pour faire la classification.

Avant de terminer cette section, nous examinerons comment la bibliothèque fastai peut vous aider à assembler vos données pour vos problèmes spécifiques.

## Questionnaire

1. What is "self-supervised learning"?
1. What is a "language model"?
1. Why is a language model considered self-supervised?
1. What are self-supervised models usually used for?
1. Why do we fine-tune language models?
1. What are the three steps to create a state-of-the-art text classifier?
1. How do the 50,000 unlabeled movie reviews help us create a better text classifier for the IMDb dataset?
1. What are the three steps to prepare your data for a language model?
1. What is "tokenization"? Why do we need it?
1. Name three different approaches to tokenization.
1. What is `xxbos`?
1. List four rules that fastai applies to text during tokenization.
1. Why are repeated characters replaced with a token showing the number of repetitions and the character that's repeated?
1. What is "numericalization"?
1. Why might there be words that are replaced with the "unknown word" token?
1. With a batch size of 64, the first row of the tensor representing the first batch contains the first 64 tokens for the dataset. What does the second row of that tensor contain? What does the first row of the second batch contain? (Careful—students often get this one wrong! Be sure to check your answer on the book's website.)
1. Why do we need padding for text classification? Why don't we need it for language modeling?
1. What does an embedding matrix for NLP contain? What is its shape?
1. What is "perplexity"?
1. Why do we have to pass the vocabulary of the language model to the classifier data block?
1. What is "gradual unfreezing"?
1. Why is text generation always likely to be ahead of automatic identification of machine-generated texts?

### Plus d'amples recherches

1. Voyez ce que vous pouvez apprendre sur les modèles de langage et la désinformation. Quels sont les meilleurs modèles linguistiques aujourd'hui? Jetez un œil à certains de leurs résultats. Les trouvez-vous convaincants? Comment un mauvais acteur pourrait-il utiliser au mieux un tel modèle pour créer des conflits et de l'incertitude?
1. Compte tenu du fait que les modèles ne seront probablement pas capables de reconnaître de manière cohérente les textes générés par la machine, quelles autres approches pourraient être nécessaires pour gérer des campagnes de désinformation à grande échelle qui tirent parti de l'apprentissage en profondeur?